In [4]:
#!pip install pandas numpy tweepy 

In [25]:
import pandas as pd #dataframe
import tweepy
from tweepy import API #searchresults
from tweepy import OAuthHandler #authenthicator
import numpy as np
import datetime # im not sure trying to get last weeks data
import os
from dateutil import parser
from tqdm.notebook import trange,tqdm

# New section

In [2]:
# personal keys and tokens from Brandons free account
consumer_key = 'OT687H4xLVN8ZBqvKEm7hAfGg'
consumer_secret = 'DP9rlNkNGZHkvKRYzCz6MdgYZWhAArZZqXGsg0UzZf1oeggNh6'
token_key = '115110276-DYsEhoByioc31EvkmDCs5jHKzKhWvT7oqxAcyJ86'
token_secret = '4kRGPgkF8NMPqs9wWLcl13jGckBuMc39jzvTySA2DT5GN'

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(token_key, token_secret)


In [3]:
# connect, second and third argument are needed to avoid timeout exception but theres still problems
api = API(auth, wait_on_rate_limit = True, wait_on_rate_limit_notify = True)

In [4]:
# get the list of files inside a path
def get_files(root,ext = '.jpg',flist =None):
    if flist is None:
        flist = []
    for root,directories,filenames in os.walk(root):
        for filename in filenames:
            if filename.endswith(ext):
                fpath = os.path.join(root,filename)
                flist.append(fpath)
    return flist

In [5]:
# query params
keywords = pd.read_csv('keywords.csv', header=0)
geolocation = "50.736455137010665,-2.2961425781250004,30mi"
today = datetime.date.today()
lastweek = (today - datetime.timedelta(days=7)).strftime('%Y-%m-%d')

In [27]:
# making query statement
queries=[]

for j in range(keywords.shape[1]):
    s=""
    for i in range(keywords.shape[0]):
        if not pd.isnull(keywords.iloc[i,j]):
            s += "\"" + keywords.iloc[i,j] + "\"" + ' OR '
    queries.append(s[:-4])


In [28]:
queries = queries[:2]
data =[]
for query in tqdm(queries): 
    for tweet in tweepy.Cursor(api.search,
                               q=query,
                               count=100,
                               result_type="recent",
                               include_entities=True,
                               lang="en",
                               geocode=geolocation,
                               since = lastweek).items():
        data.append(tweet._json)

In [29]:
my_list = []
for each_dictionary in data:
    tweet_id = each_dictionary['id']
    name = each_dictionary['user']['name']
    screen_name = each_dictionary['user']['screen_name']
    bio = each_dictionary['user']['description']
    text = each_dictionary['text']
    location = each_dictionary['user']['location']
    favorite_count = each_dictionary['favorite_count']
    retweet_count = each_dictionary['retweet_count']
    
    created_at = each_dictionary['created_at']
    datetime=parser.parse(created_at)
    date_of_tweet = datetime.strftime('%Y-%m-%d')
    time_of_tweet = datetime.strftime('%H:%M:%S')
    
    retweet_count = each_dictionary['retweet_count']
    if 'RT @' not in each_dictionary['text']:
        Tweet_type = 'Tweet'
    else:
        Tweet_type = 'ReTweet'
        
    country = []
    if each_dictionary['place'] is not None:
      Geo_location = each_dictionary['place']['name']
      country = each_dictionary['place']['country']
      

    my_list.append({'Tweet ID':str(tweet_id),
                    'Username': str(name),
                    'Tweet': str(text),
                    'Date of tweet': date_of_tweet,
                    'Time of tweet':time_of_tweet,
                    'Tweet_type':Tweet_type,
                    'Location': str(location)
                    })
    

In [30]:
df = pd.DataFrame(my_list)
df = df[df['Date of tweet']!=today.strftime('%Y-%m-%d')].reset_index(drop=True) # filtering out current date tweets
df.head()

,Tweet ID,Username,Tweet,Date of tweet,Time of tweet,Tweet_type,Location
0,1304203789885419521,marionclawson@hotmail.com,RT @DanielWBmouth: Covid Marshalls are being p...,2020-09-10,23:43:09,ReTweet,
1,1304203614278299651,nick scott,RT @DanielWBmouth: Covid Marshalls are being p...,2020-09-10,23:42:27,ReTweet,
2,1304202845839929344,Karen Ankers,RT @DanielWBmouth: Covid Marshalls are being p...,2020-09-10,23:39:24,ReTweet,Anglesey
3,1304202354603032577,⚫ Margot Williams#NHSLove #GTTO,RT @DanielWBmouth: Covid Marshalls are being p...,2020-09-10,23:37:26,ReTweet,"Ross-on-Wye, England"
4,1304201084886368257,Brian P Willmot,@stephphilip8 @realDonaldTrump @RobertKennedyJ...,2020-09-10,23:32:24,Tweet,"Bournemouth, England"


In [103]:
df.to_csv('tweet_df.csv', index=False)

In [31]:
df.shape

(5471, 7)